In [101]:
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.nn import softmax
from keras.models import Sequential 
from keras.layers import Dense, BatchNormalization, Embedding, LSTM, Flatten, Dropout, Conv1D, GlobalMaxPooling1D
from keras.metrics import Accuracy
from keras import utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np

#### Read data from csv

In [102]:
data = pd.read_csv("iotech2.csv",index_col=0)

#### Select inputs and labels

In [103]:
texts = data['Titles']
labels = data['Category']

#### Set max number of words in corpus and the max length of sentence (title).

In [104]:
NUM_WORDS = 10000
MAX_TEXT_LEN = 10
BATCH_SIZE=128

##### Create corpus

In [105]:
tokenizer =keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(texts)

In [106]:
word_count = 10
keys = list(tokenizer.word_index.keys())[:word_count]
values = list(tokenizer.word_index.values())[:word_count]
list(zip(keys, values))

[('ja', 1),
 ('f1', 2),
 ('gp', 3),
 ('2016', 4),
 ('2017', 5),
 ('auton', 6),
 ('the', 7),
 ('keskustelua', 8),
 ('onko', 9),
 ('2018', 10)]

In [107]:
sequences = tokenizer.texts_to_sequences(texts)
X = keras.utils.pad_sequences(sequences, maxlen=MAX_TEXT_LEN)  # Standardization of the number of elements in a vector
y = labels.copy()



In [108]:
print(X)

[[  0   0   0 ... 446 971 513]
 [  0   0   0 ...   0 514 972]
 [445  42 973 ... 974 106 472]
 ...
 [  0   0   0 ... 223 224 225]
 [  0   0   0 ... 226   1 227]
 [  0   0   0 ...   0  15 228]]


#### Create training, validation and testing datasets

In [109]:
X, X_test, y, y_test = train_test_split(X, y, test_size=0.1,random_state=107)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=107)
print(f'Training sample size: {len(X_train)}')
print(f'Test sample size: {len(X_test)}')
print(f'Val sample size: {len(X_val)}')

Training sample size: 1960
Test sample size: 312
Val sample size: 841


In [110]:
X_train.shape

(1960, 10)

In [111]:
len(y.unique())

5

#### Neuarl networks for classification

In [112]:
# LSTM model for classification.
model = Sequential()
model.add(Embedding(NUM_WORDS, 16, input_length=MAX_TEXT_LEN))
model.add(Dropout(0.2))
model.add(LSTM(8, return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(8))
model.add(Dropout(0.2))
model.add(Dense(5))

# CNN just for reference. Not suited for text classification
model2 = Sequential()
model2.add(Embedding(NUM_WORDS, 64, input_length=MAX_TEXT_LEN))
model2.add(Dropout(0.2))
model2.add(Conv1D(32,
                 5,
                 padding='same',
                 activation='relu',
                 strides=1))
# we use max pooling:
model2.add(GlobalMaxPooling1D())
model2.add(Dropout(0.2))
model2.add(Dense(32,activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(5))



In [113]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 10, 16)            160000    
                                                                 
 dropout_14 (Dropout)        (None, 10, 16)            0         
                                                                 
 lstm_8 (LSTM)               (None, 10, 8)             800       
                                                                 
 batch_normalization_4 (Batc  (None, 10, 8)            32        
 hNormalization)                                                 
                                                                 
 lstm_9 (LSTM)               (None, 8)                 544       
                                                                 
 dropout_15 (Dropout)        (None, 8)                 0         
                                                      

In [114]:
model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 10, 64)            640000    
                                                                 
 dropout_16 (Dropout)        (None, 10, 64)            0         
                                                                 
 conv1d_4 (Conv1D)           (None, 10, 32)            10272     
                                                                 
 global_max_pooling1d_4 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 dropout_17 (Dropout)        (None, 32)                0         
                                                                 
 dense_13 (Dense)            (None, 32)                1056      
                                                      

#### Creating callback to stop training proses when val_loss stops decreasing
#### Also creating class weights to balance data inbalances

In [115]:
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=10, verbose=1),  
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, min_delta=0.01, min_lr=1e-10, patience=3, verbose=1, mode='auto')
]
class_weights={0:1.,1:1.,2:1.,3:1.,4:1.,5:1}

In [116]:
model.compile(metrics=['accuracy'], loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='rmsprop')

#### Training the model

In [117]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=40, class_weight=class_weights, callbacks=callbacks,validation_data=(X_val,y_val))

Epoch 1/40
16/16 [==============================] - 3s 47ms/step - loss: 1.5020 - accuracy: 0.4112 - val_loss: 1.5733 - val_accuracy: 0.4554 - lr: 0.0010
Epoch 2/40
16/16 [==============================] - 0s 9ms/step - loss: 1.3227 - accuracy: 0.5750 - val_loss: 1.5573 - val_accuracy: 0.4614 - lr: 0.0010
Epoch 3/40
16/16 [==============================] - 0s 9ms/step - loss: 1.1533 - accuracy: 0.6449 - val_loss: 1.5392 - val_accuracy: 0.3722 - lr: 0.0010
Epoch 4/40
16/16 [==============================] - 0s 9ms/step - loss: 1.0204 - accuracy: 0.7015 - val_loss: 1.5195 - val_accuracy: 0.5291 - lr: 0.0010
Epoch 5/40
16/16 [==============================] - 0s 8ms/step - loss: 0.9022 - accuracy: 0.7429 - val_loss: 1.4967 - val_accuracy: 0.4946 - lr: 0.0010
Epoch 6/40
16/16 [==============================] - 0s 8ms/step - loss: 0.8025 - accuracy: 0.7679 - val_loss: 1.4744 - val_accuracy: 0.5339 - lr: 0.0010
Epoch 7/40
16/16 [==============================] - 0s 8ms/step - loss: 0.7223 - 

#### Crating some function to tests the trained model

In [118]:
class_names=[
"Prosessorit, ylikellotus, emolevyt ja muistit",
"Näytönohjaimet",
"Tallennus",
"Kotelot ja virtalähteet",
"Jäähdytys",
"Konepaketit",
"Kannettavat tietokoneet",
"Buildit, setupit, kotelomodifikaatiot & DIY",
"Oheislaitteet ja muut PC-komponentit"
]
class_names2=[
"Autot ja liikenne",
"Urheilu",
"TV- & nettisarjat, elokuvat ja musiikki",
"Ruoka & juoma",
"Koti ja asuminen",
]

In [119]:
class_names2

['Autot ja liikenne',
 'Urheilu',
 'TV- & nettisarjat, elokuvat ja musiikki',
 'Ruoka & juoma',
 'Koti ja asuminen']

In [120]:
predictions=model.predict(X_test)
score = softmax(predictions[0])
print(
    "This text most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names2[np.argmax(score)], 100 * np.max(score))
)


10/10 [==============================] - 1s 2ms/step
This text most likely belongs to TV- & nettisarjat, elokuvat ja musiikki with a 97.25 percent confidence.


In [121]:
print(predictions[0])

[-0.30520305 -1.899174    4.902777   -2.5772963   1.0474505 ]


In [122]:
def ml_pipeline(text: str,ml_model) -> str:
    """LSTM model prediction function for this sample"""
    try:
        sequence = tokenizer.texts_to_sequences([text])
        sequence = keras.utils.pad_sequences(sequence, maxlen=MAX_TEXT_LEN)
        if sequence.max() == 0:
            return 'Enter the words in English'
        else:
            predict = ml_model.predict(sequence, verbose=0)
            score = softmax(predict)
            print(predict)
            return class_names2[np.argmax(score)]
        
    except AttributeError as AE:
        print(AE)
        return 'Enter the text'



In [128]:
 ml_pipeline('Harvemmin ostan kasviksia, mutta jos ostan niin olutta',model)

[[-1.0146415   0.22125839 -0.4810922   2.1169562   2.0607998 ]]


'Ruoka & juoma'

In [124]:
data['Category'].value_counts()

3    893
0    873
2    581
1    404
4    362
Name: Category, dtype: int64